##### Social Media Analytics
### Introduction to Text Mining
## Named Entity Recognition

### Initial setup

In [1]:
import warnings

warnings.filterwarnings("ignore")
import nltk

nltk.download("punkt")

[nltk_data] Downloading package punkt to /home/tosk/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
from nltk.tokenize import sent_tokenize
from tqdm import tqdm

In [4]:
# Import packages
import csv
import re
from collections import Counter

import nltk
import numpy
import numpy as np
import pandas as pd
import spacy
from bs4 import BeautifulSoup
from spacy import displacy

In [5]:
ds = pd.read_parquet("DailyMail.parquet.snappy", engine="fastparquet")

### Functions

In [6]:
# Text preprocessing
def textPreProcess(
    rawText,
    removeHTML=True,
    charsToRemove=r"\?|\.|\!|\;|\.|\"|\,|\(|\)|\&|\:|\-",
    removeNumbers=True,
    removeLineBreaks=False,
    specialCharsToRemove=r"[^\x00-\xfd]",
    convertToLower=True,
    removeConsecutiveSpaces=True,
):
    if type(rawText) != str:
        return rawText
    procText = rawText

    # Remove HTML
    if removeHTML:
        procText = BeautifulSoup(procText, "html.parser").get_text()

    # Remove punctuation and other special characters
    if len(charsToRemove) > 0:
        procText = re.sub(charsToRemove, " ", procText)

    # Remove numbers
    if removeNumbers:
        procText = re.sub(r"\d+", " ", procText)

    # Remove line breaks
    if removeLineBreaks:
        procText = procText.replace("\n", " ").replace("\r", "")

    # Remove special characters
    if len(specialCharsToRemove) > 0:
        procText = re.sub(specialCharsToRemove, " ", procText)

    # Normalize to lower case
    # Since we want to find the persons in the articles we don' want to normalize to lower case

    # Replace multiple consecutive spaces with just one space
    if removeConsecutiveSpaces:
        procText = re.sub(" +", " ", procText)

    return procText

### Analysis

In [7]:
# Create a dataframe with only the description
processedReviews = pd.DataFrame(
    data=ds.text.apply(
        textPreProcess, charsToRemove="", removeLineBreaks=False, removeNumbers=False
    ).values,
    index=ds.index,
    columns=["PreProcessedText"],
)

In [8]:
# Remove rows with empty text
processedReviews.PreProcessedText = processedReviews.PreProcessedText.str.strip()
processedReviews = processedReviews[processedReviews.PreProcessedText != ""]

In [9]:
processedReviews.head()

,PreProcessedText
0,A wave of looting has reportedly hit a border ...
1,"In the dawn hours of May 3, Moscovites awoke t..."
2,Visitors to St. Peter's Basilica got an unexpe...
3,Russian security services have accused Apple o...
4,A foster mum who rescued orphans from war-torn...


In [10]:
sentences = []
for review in processedReviews["PreProcessedText"]:
    sentences = sentences + nltk.tokenize.sent_tokenize(review)

In [11]:
# Load Spacy English model
nlp = spacy.load("en_core_web_sm")

In [12]:
# creating a dataframe with each article and the persons mentioned
persons_df = pd.DataFrame({"id": [], "persons": []})

for i, sentence in enumerate(tqdm(sentences)):
    doc = nlp(sentence)
    entities = [(X.text, X.label_) for X in doc.ents]
    persons = []
    for entity in entities:
        if entity[1] == "PERSON":
            persons.append(entity[0])

    persons_df.loc[i] = [i, persons]

100%|██████████| 28576/28576 [26:00<00:00, 18.31it/s]   


In [13]:
persons_df

,id,persons
0,0,[Vladimir Putin's]
1,1,[Putin]
2,2,[]
3,3,[]
4,4,[]
...,...,...
28571,28571,[]
28572,28572,[]
28573,28573,[Putin]
28574,28574,[Putin]


In [14]:
# cleaning the dataframe
persons_df_clean = pd.DataFrame({"id": [], "persons": []})

# taking just the last name from all the entities that has both first name and second name
for i in tqdm(range(len(persons_df))):
    persons = persons_df.loc[i][1]
    cleaned_persons = []
    for person in persons:
        person = person.split(" ")[-1]
        cleaned_persons.append(person)

    persons_df_clean.loc[i] = [i, cleaned_persons]

100%|██████████| 28576/28576 [00:52<00:00, 545.95it/s] 


In [15]:
final_persons_df = pd.DataFrame({"id": [], "persons": []})

for i in range(len(persons_df_clean)):
    final_persons_df.loc[i] = [i, list(set(persons_df_clean.loc[i][1]))]

In [16]:
persons_df_clean

,id,persons
0,0,[Putin's]
1,1,[Putin]
2,2,[]
3,3,[]
4,4,[]
...,...,...
28571,28571,[]
28572,28572,[]
28573,28573,[Putin]
28574,28574,[Putin]


In [18]:
final_persons_df
edge_dataframe = pd.DataFrame({"Source": [], "Target": []})

for i in tqdm(range(len(final_persons_df))):
    persons = final_persons_df.loc[i][1]
    for j in range(len(persons) - 1):
        for k in range(j + 1, len(persons)):
            values_to_add = {"Source": persons[j], "Target": persons[k]}
            row_to_add = pd.Series(values_to_add)
            edge_dataframe = edge_dataframe.append(row_to_add, ignore_index=True)

  0%|          | 20/28576 [00:00<00:10, 2762.77it/s]


AttributeError: 'DataFrame' object has no attribute 'append'

In [ ]:
for i, row in edge_dataframe.iterrows():
    if row[0] == row[1]:
        edge_dataframe = edge_dataframe.drop(i)

In [ ]:
edge_dataframe.head(50)

In [ ]:
# function to check if there is a number in a string
def num_there(s):
    return any(i.isdigit() for i in s)

In [ ]:
for i, row in edge_dataframe.iterrows():

    if row[0] == row[1]:
        edge_dataframe = edge_dataframe.drop(i)

    elif row[0] == row[1]:
        edge_dataframe = edge_dataframe.drop(i)

    elif num_there(row[0]) == True or num_there(row[1]) == True:
        edge_dataframe = edge_dataframe.drop(i)

    elif len(row[0]) == 1 or len(row[1]) == 1:
        edge_dataframe = edge_dataframe.drop(i)

    elif "." in row[0] or "." in row[1]:
        edge_dataframe = edge_dataframe.drop(i)

    elif "." in row[0] or "." in row[1]:
        edge_dataframe = edge_dataframe.drop(i)

    elif row[0] == "Kyiv" or row[1] == "Kyiv":
        edge_dataframe = edge_dataframe.drop(i)

    elif "Peter" in row[0] or "Peter" in row[1]:
        edge_dataframe = edge_dataframe.drop(i)

    elif "Twitter" in row[0] or "Twitter" in row[1]:
        edge_dataframe = edge_dataframe.drop(i)

    elif "?" in row[0] or "?" in row[1]:
        edge_dataframe = edge_dataframe.drop(i)

    elif '"' in row[0] or '"' in row[1]:
        edge_dataframe = edge_dataframe.drop(i)

In [ ]:
edge_dataframe.head(10)

In [27]:
edges_DailyMail = pd.DataFrame({"Source": [], "Target": []})

for i in tqdm(range(len(edge_dataframe))):

    person1 = edge_dataframe.iloc[i][0]
    person2 = edge_dataframe.iloc[i][1]

    person1 = person1.split("-")[-1]
    person2 = person2.split("-")[-1]

    if "'" in person1:
        person1 = person1.split("'")[0]

    if "'" in person2:
        person2 = person2.split("'")[0]

    edges_DailyMail.loc[i] = [person1, person2]

100%|██████████| 13185/13185 [00:32<00:00, 401.55it/s]


In [ ]:
edges_DailyMail.iloc[515]

In [ ]:
edges_DailyMail.to_csv("edges_DailyMail.csv", index=False)